Connect through SSH to jetbot and run:

cd workspace/jetbot/notebooks/objdet_custom/

sudo gst-launch-1.0 nvarguscamerasrc num-buffers=400 ! 'video/x-raw(memory:NVMM),width=1280, height=720, framerate=120/1, format=NV12' ! omxh264enc ! qtmux ! filesink location=recordings_10_06/test.mp4 -e

to change the duration of the video frame increase/dec the num-buffers

To change the brightness and contrast of the image, use the below command:

sudo gst-launch-1.0 nvarguscamerasrc num-buffers=2000 ! 'video/x-raw(memory:NVMM),width=1280, height=720, framerate=120/1, format=NV12' ! nvvidconv ! videobalance contrast= brightness= ! nvvidconv ! omxh264enc ! qtmux ! filesink location=test.mp4 -e

gst-inspect-1.0 nvarguscamerasrc

 tnr-mode            : property to select temporal noise reduction mode
                        flags: readable, writable
                        Enum "GstNvArgusCamTNRMode" Default: 1, "NoiseReduction_Fast"
                           (0): NoiseReduction_Off - GST_NVCAM_NR_OFF
                           (1): NoiseReduction_Fast - GST_NVCAM_NR_FAST
                           (2): NoiseReduction_HighQuality - GST_NVCAM_NR_HIGHQUALITY


  tnr-strength        : property to adjust temporal noise reduction strength
                        flags: readable, writable
                        Float. Range:              -1 -               1 Default:              -1


sudo gst-launch-1.0 nvarguscamerasrc num-buffers=400 tnr-mode=2 tnr-strength=1 ! 'video/x-raw(memory:NVMM),width=1280, height=720, framerate=120/1, format=NV12' ! omxh264enc ! qtmux ! filesink location=recordings_10_06/test1.mp4 -e


gst-inspect-1.0 videobalance

                        flags: readable, writable
                        String. Default: "videobalance0"
  parent              : The parent of the object
                        flags: readable, writable
                        Object of type "GstObject"
  qos                 : Handle Quality-of-Service events
                        flags: readable, writable
                        Boolean. Default: true
  contrast            : contrast
                        flags: readable, writable, controllable
                        Double. Range:               0 -               2 Default:               1
  brightness          : brightness
                        flags: readable, writable, controllable
                        Double. Range:              -1 -               1 Default:               0
  hue                 : hue
                        flags: readable, writable, controllable
                        Double. Range:              -1 -               1 Default:               0
  saturation          : saturation
                        flags: readable, writable, controllable
                        Double. Range:               0 -               2 Default:               1

Tests:

sudo gst-launch-1.0 nvarguscamerasrc num-buffers=400 ! 'video/x-raw(memory:NVMM),width=1280, height=720, framerate=120/1, format=NV12' ! nvvidconv ! videobalance contrast=2 brightness=-0.15 ! nvvidconv ! omxh264enc ! qtmux ! filesink location=recordings_10_06/test2.mp4 -e

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [1]:
import ipywidgets.widgets as widgets

controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [4]:
from jetbot import Robot
import traitlets

robot = Robot()

left_link = traitlets.dlink((controller.axes[1], 'value'), (robot.left_motor, 'value'), transform=lambda x: -0.2*x)
right_link = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.2*x)